In [68]:
import pandas as pd
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from qdrant_client.models import Distance, VectorParams, PointStruct,Batch,Payload
import uuid

In [69]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
client = QdrantClient(url="http://localhost:6333")

if (client.collection_exists(collection_name="no_coordinates_collection")):
    client.delete_collection(collection_name="no_coordinates_collection")
    
collection_name = "no_coordinates_collection"

/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [70]:
# Replace 'path_to_yelp_academic_dataset_business.json' with the actual path to your JSON file
features = ['business_id', 'name', 'address', 'city','postal_code', 'categories', 'attributes', 'longitude', 'latitude', 'stars', 'review_count', 'hours']
df = pd.read_json('archive/yelp_academic_dataset_business.json', lines=True)
df_cali= df[df['state'] == 'CA']
df_features = df_cali[features]

df_features

,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"Doctors, Traditional Chinese Medicine, Naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"Women's Clothing, Accessories, Children's Clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'..."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"Towing, Automotive, Body Shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."
120,bYjnX_J1bHZob10DoSFkqQ,Tinkle Belle Diaper Service,,Santa Barbara,93101,"Parenting Classes, Maternity Wear, Fashion, Ed...","{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",-119.710749,34.420334,5.0,17,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"Event Planning & Services, Wine Tasting Room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"Restaurants, Burgers, Food, Ice Cream & Frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"Shopping, Eyewear & Opticians, Health & Medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '..."
150290,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,93105,"Restaurants, Pizza, Food","{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...",-119.739681,34.440689,4.0,299,"{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'..."


In [71]:
# Concatenating address, city, and postal code with a comma and space
df_features['full_address'] = df_features.apply(lambda x: f"{x['address']} {x['postal_code']}", axis=1)
print(len(df_features))
df_features.replace('', np.nan, inplace=True)
df_features.dropna(subset=['address', 
                           'city',
                           'postal_code',
                           'categories'
                           ],
                    inplace=True
                        )
print(len(df_features))
df_features['categories'].apply(lambda x: [item.lower() for item in x])
df_features


5203
4715


/tmp/ipykernel_30462/783486201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['full_address'] = df_features.apply(lambda x: f"{x['address']} {x['postal_code']}", axis=1)
/tmp/ipykernel_30462/783486201.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features.replace('', np.nan, inplace=True)
/tmp/ipykernel_30462/783486201.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours,full_address
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"Doctors, Traditional Chinese Medicine, Naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 93101"
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"Women's Clothing, Accessories, Children's Clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St 93101
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C 93101"
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"Towing, Automotive, Body Shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St 93103
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,93101,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'OutdoorSeating': 'True', 'RestaurantsAttire'...",-119.685019,34.408715,4.0,2404,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",230 Stearns Wharf 93101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150239,TSwMwVq5GtQh5LW2t32uGA,Woody's Roundup Bar & Grill,"Earl Warren Showgrounds, 3400 Calle Real",Santa Barbara,93105,"Barbeque, Bars, Restaurants, Nightlife","{'Caters': 'True', 'BusinessParking': '{'garag...",-119.735645,34.430895,2.0,6,None,"Earl Warren Showgrounds, 3400 Calle Real 93105"
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"Event Planning & Services, Wine Tasting Room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St 93105
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"Restaurants, Burgers, Food, Ice Cream & Frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar 93117
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"Shopping, Eyewear & Opticians, Health & Medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B 93101"


In [72]:
df_features['categories']= df_features['categories'].apply(lambda x: x.lower() )
df_features

/tmp/ipykernel_30462/3198185297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['categories']= df_features['categories'].apply(lambda x: x.lower() )


,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours,full_address
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"doctors, traditional chinese medicine, naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 93101"
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"women's clothing, accessories, children's clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St 93101
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"food, restaurants, salad, coffee & tea, breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C 93101"
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"towing, automotive, body shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St 93103
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,93101,"live/raw food, restaurants, seafood, beer bar,...","{'OutdoorSeating': 'True', 'RestaurantsAttire'...",-119.685019,34.408715,4.0,2404,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",230 Stearns Wharf 93101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150239,TSwMwVq5GtQh5LW2t32uGA,Woody's Roundup Bar & Grill,"Earl Warren Showgrounds, 3400 Calle Real",Santa Barbara,93105,"barbeque, bars, restaurants, nightlife","{'Caters': 'True', 'BusinessParking': '{'garag...",-119.735645,34.430895,2.0,6,None,"Earl Warren Showgrounds, 3400 Calle Real 93105"
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"event planning & services, wine tasting room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St 93105
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"restaurants, burgers, food, ice cream & frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar 93117
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"shopping, eyewear & opticians, health & medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B 93101"


In [73]:

# Prepare an empty list to store combined embeddings
combined_embeddings = []

# Loop through each row in the DataFrame
for index, row in tqdm(df_features.iterrows(), total=df_features.shape[0], desc="Encoding rows"):
    # Generate embeddings for each row
    address_embedding = model.encode(row['full_address'])
    coordinates = np.array([row['longitude'], row['latitude']])
    # combined_embedding = np.concatenate((address_embedding, coordinates))
    combined_embeddings.append(address_embedding)

df_features['combined_embeddings'] = combined_embeddings
df_features

Encoding rows: 100%|██████████| 4715/4715 [00:37<00:00, 125.82it/s]
/tmp/ipykernel_30462/1002732970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['combined_embeddings'] = combined_embeddings


,business_id,name,address,city,postal_code,categories,attributes,longitude,latitude,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,93101,"doctors, traditional chinese medicine, naturop...",{'ByAppointmentOnly': 'True'},-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 93101","[-0.06993202, 0.0143677825, -0.047251716, -0.0..."
26,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,93101,"women's clothing, accessories, children's clot...","{'BusinessAcceptsCreditCards': 'True', 'BikePa...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St 93101,"[-0.09852362, -0.018569747, -0.011499168, 0.04..."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,93101,"food, restaurants, salad, coffee & tea, breakf...","{'RestaurantsTakeOut': 'True', 'NoiseLevel': '...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C 93101","[-0.0004515124, 0.02757613, -0.02448914, -0.03..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,825 Cacique St,Santa Barbara,93103,"towing, automotive, body shops","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St 93103,"[-0.06591021, 0.043082647, -0.039819557, -0.00..."
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,93101,"live/raw food, restaurants, seafood, beer bar,...","{'OutdoorSeating': 'True', 'RestaurantsAttire'...",-119.685019,34.408715,4.0,2404,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",230 Stearns Wharf 93101,"[-0.015296874, -0.056106303, -0.042750373, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150239,TSwMwVq5GtQh5LW2t32uGA,Woody's Roundup Bar & Grill,"Earl Warren Showgrounds, 3400 Calle Real",Santa Barbara,93105,"barbeque, bars, restaurants, nightlife","{'Caters': 'True', 'BusinessParking': '{'garag...",-119.735645,34.430895,2.0,6,None,"Earl Warren Showgrounds, 3400 Calle Real 93105","[-0.15032059, -0.016469747, -0.03390762, -0.12..."
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,2012 De La Vina St,Santa Barbara,93105,"event planning & services, wine tasting room, ...","{'BusinessParking': '{'garage': False, 'street...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St 93105,"[-0.05024828, 0.070451386, -0.040242776, -0.03..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,93117,"restaurants, burgers, food, ice cream & frozen...","{'OutdoorSeating': 'True', 'BusinessAcceptsBit...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar 93117,"[-0.056329798, 0.05286719, -0.03173885, -0.042..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","1805 State St, Ste B",Santa Barbara,93101,"shopping, eyewear & opticians, health & medica...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B 93101","[-0.044830225, 0.019696986, -0.026411047, -0.0..."


In [74]:
response = client.create_collection(
    collection_name=f"{collection_name}",
    vectors_config=VectorParams(size=len(df_features['combined_embeddings'][0]), distance=Distance.COSINE),
)
print(response)
client.get_collection(collection_name)

True


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), pay

In [75]:
df_features = df_features.drop(columns=['address', 'city', 'postal_code','attributes'])
df_features

,business_id,name,categories,longitude,latitude,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","doctors, traditional chinese medicine, naturop...",-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 93101","[-0.06993202, 0.0143677825, -0.047251716, -0.0..."
26,noByYNtDLQAra9ccqxdfDw,H&M,"women's clothing, accessories, children's clot...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St 93101,"[-0.09852362, -0.018569747, -0.011499168, 0.04..."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"food, restaurants, salad, coffee & tea, breakf...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C 93101","[-0.0004515124, 0.02757613, -0.02448914, -0.03..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,"towing, automotive, body shops",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St 93103,"[-0.06591021, 0.043082647, -0.039819557, -0.00..."
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,"live/raw food, restaurants, seafood, beer bar,...",-119.685019,34.408715,4.0,2404,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",230 Stearns Wharf 93101,"[-0.015296874, -0.056106303, -0.042750373, -0...."
...,...,...,...,...,...,...,...,...,...,...
150239,TSwMwVq5GtQh5LW2t32uGA,Woody's Roundup Bar & Grill,"barbeque, bars, restaurants, nightlife",-119.735645,34.430895,2.0,6,None,"Earl Warren Showgrounds, 3400 Calle Real 93105","[-0.15032059, -0.016469747, -0.03390762, -0.12..."
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,"event planning & services, wine tasting room, ...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St 93105,"[-0.05024828, 0.070451386, -0.040242776, -0.03..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,"restaurants, burgers, food, ice cream & frozen...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar 93117,"[-0.056329798, 0.05286719, -0.03173885, -0.042..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","shopping, eyewear & opticians, health & medica...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B 93101","[-0.044830225, 0.019696986, -0.026411047, -0.0..."


In [76]:
print('dimensions',len(df_features['combined_embeddings'][0]))

dimensions 384


In [77]:
df_features

,business_id,name,categories,longitude,latitude,stars,review_count,hours,full_address,combined_embeddings
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","doctors, traditional chinese medicine, naturop...",-119.711197,34.426679,5.0,7,None,"1616 Chapala St, Ste 2 93101","[-0.06993202, 0.0143677825, -0.047251716, -0.0..."
26,noByYNtDLQAra9ccqxdfDw,H&M,"women's clothing, accessories, children's clot...",-119.700460,34.420209,3.0,24,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",827-833 State St 93101,"[-0.09852362, -0.018569747, -0.011499168, 0.04..."
85,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"food, restaurants, salad, coffee & tea, breakf...",-119.690672,34.414445,4.0,389,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '...","131 Anacapa St, Ste C 93101","[-0.0004515124, 0.02757613, -0.02448914, -0.03..."
91,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,"towing, automotive, body shops",-119.677032,34.419620,4.5,32,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...",825 Cacique St 93103,"[-0.06591021, 0.043082647, -0.039819557, -0.00..."
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,"live/raw food, restaurants, seafood, beer bar,...",-119.685019,34.408715,4.0,2404,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",230 Stearns Wharf 93101,"[-0.015296874, -0.056106303, -0.042750373, -0...."
...,...,...,...,...,...,...,...,...,...,...
150239,TSwMwVq5GtQh5LW2t32uGA,Woody's Roundup Bar & Grill,"barbeque, bars, restaurants, nightlife",-119.735645,34.430895,2.0,6,None,"Earl Warren Showgrounds, 3400 Calle Real 93105","[-0.15032059, -0.016469747, -0.03390762, -0.12..."
150264,Bo-GGwQNcLVwyKeYkbfAYg,Vino Divino,"event planning & services, wine tasting room, ...",-119.717222,34.429154,5.0,19,"{'Monday': '0:0-0:0', 'Tuesday': '12:0-18:0', ...",2012 De La Vina St 93105,"[-0.05024828, 0.070451386, -0.040242776, -0.03..."
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,"restaurants, burgers, food, ice cream & frozen...",-119.856886,34.412966,2.5,12,None,900 Embarcadero Del Mar 93117,"[-0.056329798, 0.05286719, -0.03173885, -0.042..."
150284,f7rTfTv6dwO-ovKhz-Rvgw,"Joanne & Tem Gronquist, OD","shopping, eyewear & opticians, health & medica...",-119.712718,34.428995,4.5,15,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...","1805 State St, Ste B 93101","[-0.044830225, 0.019696986, -0.026411047, -0.0..."


In [78]:

def hash_to_uuid(unique_string):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, unique_string))

def process_catagoires(categories):
    return categories.split(', ') if categories else []
# Adjust the Payload construction if needed
def create_point(row):
    return PointStruct(
        id=hash_to_uuid(row['business_id']),
        vector=np.array(row['combined_embeddings']).tolist(),  # Ensure it's converted to list properly
        payload={
            'name' : row['name'],
            'categories' : process_catagoires(row['categories']) ,  # Assuming categories is a single string of categories
            'stars' : float(row['stars']),  # Ensure it's a float
            'review_count' : int(row['review_count']),  # Ensure it's an int
            'hours' : row['hours'],  # Ensure hours are in a JSON serializable format
            'full_address': row['full_address'],
            'latitude': float(row['latitude']),
            'longitude': float(row['longitude'])
        }
        )

points = []
for index, row in tqdm(df_features.iterrows()):   
    point = create_point(row)
    points.append(point)


print(len(points))



4715it [00:00, 13575.26it/s]

4715


In [79]:
print(points[0])

id='92fc196f-0e3c-5f24-9ebb-09b821e99fce' vector=[-0.06993202120065689, 0.014367782510817051, -0.04725171625614166, -0.020355550572276115, -0.1145583912730217, -0.05246223509311676, -0.07413360476493835, 0.022925402969121933, 0.0067716618068516254, -0.0054024746641516685, 0.04939054697751999, -0.046318620443344116, -0.03806042671203613, -0.10681832581758499, 0.009754921309649944, -0.07648216187953949, 0.04417833313345909, -0.03486352041363716, 0.07758472859859467, 0.015389762818813324, 0.08016222715377808, 0.06545956432819366, -0.05271133407950401, -0.011816699989140034, 0.0015049620997160673, 0.06447230279445648, 0.02769685909152031, 0.038753967732191086, 0.009488199837505817, -0.06097801402211189, 0.06240889057517052, -0.040337253361940384, 0.0016662287525832653, 0.022141797468066216, 0.010518930852413177, 0.030841711908578873, -0.009245544672012329, -0.00483012804761529, 0.12020358443260193, -0.021445393562316895, -0.022279204800724983, 0.010369743220508099, 0.006014680024236441, -0

In [80]:

def upload_batches(points, batch_size=100):

    total_batches = len(points) // batch_size + (1 if len(points) % batch_size > 0 else 0)

    for i in tqdm(range(total_batches), desc="Uploading batches"):
        start_index = i * batch_size
        end_index = start_index + batch_size
        batch_points = points[start_index:end_index]
 
        response = client.upsert(
            collection_name=f"{collection_name}",
            points=batch_points
        )
        # print(f"Batch {i+1}/{total_batches} uploaded. Response: {response}")

    

In [81]:
upload_batches(points, batch_size=100)


Uploading batches:  15%|█▍        | 7/48 [00:00<00:00, 69.76it/s]

Batch 1/48 uploaded. Response: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 2/48 uploaded. Response: operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 3/48 uploaded. Response: operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 4/48 uploaded. Response: operation_id=3 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 5/48 uploaded. Response: operation_id=4 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 6/48 uploaded. Response: operation_id=5 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 7/48 uploaded. Response: operation_id=6 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 8/48 uploaded. Response: operation_id=7 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 9/48 uploaded. Response: operation_id=8 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 10/48 uploaded. Response: operation_id=9 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 11/48 uploaded. Response: operation_id=10 status=<UpdateStatus.COMPLETED: 'complete

Uploading batches:  29%|██▉       | 14/48 [00:00<00:00, 69.28it/s]

Batch 13/48 uploaded. Response: operation_id=12 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 14/48 uploaded. Response: operation_id=13 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 15/48 uploaded. Response: operation_id=14 status=<UpdateStatus.COMPLETED: 'completed'>


Uploading batches:  46%|████▌     | 22/48 [00:00<00:00, 70.67it/s]

...

Batch 28/48 uploaded. Response: operation_id=27 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 29/48 uploaded. Response: operation_id=28 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 30/48 uploaded. Response: operation_id=29 status=<UpdateStatus.COMPLETED: 'completed'>


Uploading batches:  79%|███████▉  | 38/48 [00:00<00:00, 70.25it/s]

Batch 31/48 uploaded. Response: operation_id=30 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 32/48 uploaded. Response: operation_id=31 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 33/48 uploaded. Response: operation_id=32 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 34/48 uploaded. Response: operation_id=33 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 35/48 uploaded. Response: operation_id=34 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 36/48 uploaded. Response: operation_id=35 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 37/48 uploaded. Response: operation_id=36 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 38/48 uploaded. Response: operation_id=37 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 39/48 uploaded. Response: operation_id=38 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 40/48 uploaded. Response: operation_id=39 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 41/48 uploaded. Response: operation_id=40 status=<UpdateStatus.C

Uploading batches: 100%|██████████| 48/48 [00:00<00:00, 70.62it/s]

Batch 46/48 uploaded. Response: operation_id=45 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 47/48 uploaded. Response: operation_id=46 status=<UpdateStatus.COMPLETED: 'completed'>
Batch 48/48 uploaded. Response: operation_id=47 status=<UpdateStatus.COMPLETED: 'completed'>


In [100]:
random_cell = df_features[['full_address','latitude','longitude']].sample(n=5)
print(random_cell)


                 full_address   latitude   longitude
104747    835 Laguna St 93101  34.424210 -119.695857
126683  612 N Milpas St 93103  34.427684 -119.686886
5913       214 State St 93101  34.414556 -119.691868
123409  900 N Milpas St 93103  34.430249 -119.690410
38666   6558 Pardall Rd 93117  34.413288 -119.856488
